In [29]:
!pip install boto3
import boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.0 MB/s eta 0:00:000:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 2.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 4.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import sys
sys.path.append("../")

import argparse
import datetime
import json
import os
import re
from time import sleep


from sshtunnel import SSHTunnelForwarder
import pymysql

import numpy as np
import pandas as pd
import requests

from vook_db_v7.config import MAX_PAGE, REQ_URL, WANT_ITEMS, req_params
from vook_db_v7.local_config import *

# from logzero import logger

In [5]:
"""DBからテーブル取得"""

config_ec2 = get_ec2_config()
query = """
SELECT
    *
FROM
    products 
"""
df_from_db = pd.DataFrame()

# SSHトンネルの設定
with SSHTunnelForwarder(
    (config_ec2["host_name"], config_ec2["ec2_port"]),
    ssh_username=config_ec2["ssh_username"],
    ssh_pkey=config_ec2["ssh_pkey"],
    remote_bind_address=(
        config_ec2["rds_end_point"],
        config_ec2["rds_port"],
    ),
) as server:
    print(f"Local bind port: {server.local_bind_port}")
    conn = None

    try:
        conn = pymysql.connect(
            **get_rds_config(server.local_bind_port), connect_timeout=10
        )
        cursor = conn.cursor()
        # SQLクエリの実行
        cursor.execute(query)
        for row in cursor:  # column1, column2, ...は取得したいカラム名に合わせて変更してください
            #print(row)
            
            df_from_db = pd.concat([df_from_db, pd.DataFrame([row])], ignore_index=True)

    except pymysql.MySQLError as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if conn is not None:
            conn.close()

Local bind port: 50184


In [27]:
df_from_db
# 新しいデータ
new_data = {'id': 9999999, 'name': 'test', 'url': 'test', 'price': 9999999, 'knowledge_id': 88, 'platform_id': 1, 'size_id': 999,
           'created_at': '2023-11-19 13:14:08.081758', 'updated_at': '2023-11-19 13:14:08.081758'}
df_new = pd.concat([df_from_db, pd.DataFrame([new_data])], ignore_index=True)
df_new.tail()

,id,name,url,price,knowledge_id,platform_id,size_id,created_at,updated_at
2702,3056,【中古】Lee◆カバーオール/34/ナイロン/カーキ/91-J【メンズウェア】,https://hb.afl.rakuten.co.jp/hgc/g00r3cee.brn1...,4290,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2703,3057,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00qs41e.brn1...,440,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2704,3058,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00svqpe.brn1...,440,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2705,3059,【中古】 Brenda Lee / Here Comes That Feeling 輸入盤 ...,https://hb.afl.rakuten.co.jp/hgc/g00tbz7e.brn1...,390,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758
2706,9999999,test,test,9999999,88,1,999,2023-11-19 13:14:08.081758,2023-11-19 13:14:08.081758


In [35]:
# EC2インスタンスの情報
ec2_instance = {
    'host': '54.199.201.208',  # EC2のパブリックIPアドレス
    'user': 'ec2-user',
    'key_file': '~/.ssh/vook-rails-ssh-key.pem'  # EC2キーペアの秘密鍵ファイルのパス
}

# RDSの情報
rds_instance = {
    'host': 'vook-rails-db-dev-1.ctutkiavfpne.ap-northeast-1.rds.amazonaws.com',  # RDSのエンドポイント
    'user': 'root',
    'password': 'rds-vook',
    'db_name': 'vook_web_v3_development',
    "charset": "utf8mb4",
    "cursorclass": pymysql.cursors.DictCursor,
}

# SSHトンネルの設定
with SSHTunnelForwarder(
    (ec2_instance['host'], 22),
    ssh_username=ec2_instance['user'],
    ssh_pkey=ec2_instance['key_file'],
    remote_bind_address=(rds_instance['host'], 3306)
) as tunnel:
    # RDSへの接続設定
    conn = pymysql.connect(
        host='127.0.0.1',
        port=tunnel.local_bind_port,
        user=rds_instance['user'],
        password=rds_instance['password'],
        database=rds_instance['db_name'],
        charset=rds_instance['charset'],
        cursorclass=rds_instance['cursorclass'],
        
    )

    # データを格納する処理（以下は例）
    # ここにデータをRDSに格納するための具体的な処理を書く

    # 接続を閉じる
    conn.close()

2023-11-19 17:49:59,966| ERROR   | Could not establish connection from local ('127.0.0.1', 50462) to remote ('vook-rails-db-dev-1.ctutkiavfpne.ap-northeast-1.rds.amazonaws.com', 3306) side of the tunnel: open new channel ssh error: Timeout opening channel.


OperationalError: (2013, 'Lost connection to MySQL server during query')

In [ ]:
 "host_name": "3.114.154.242",
        "ec2_port": 22,
        "ssh_username": "ec2-user",
        "ssh_pkey": "~/.ssh/vook-rails-ssh-key.pem",
        "rds_end_point": "vook-rails-db.ctutkiavfpne.ap-northeast-1.rds.amazonaws.com",
        "rds_port": 3306,